<a href="https://colab.research.google.com/github/Playmen998/Optimizing_SUM_Bag/blob/main/%20Optimizing_SUM_Bag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install gekko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pandas as pd
import numpy as np
import itertools
from gekko import GEKKO

https://docs.google.com/spreadsheets/d/1nsDYLaWCKBYYWZUwiwLZ55sOi8pyn9xW/edit?usp=share_link&ouid=109163141300161595709&rtpof=true&sd=true

In [39]:
#Загружаем таблицу
!gdown --id 1nsDYLaWCKBYYWZUwiwLZ55sOi8pyn9xW

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1nsDYLaWCKBYYWZUwiwLZ55sOi8pyn9xW
To: /content/vseKAtest.xlsx
100% 40.0k/40.0k [00:00<00:00, 99.1MB/s]


Необходимо решить задачу оптимизации. Нужно подобрать таких **клиентов**, чтобы сумма их платежей соответствовала условиям каждой группы.  
По условию задачи каждый портфель (группа) должна иметь сумму платежей не больше определенного процента от всей суммы, так например:
- ЩЭВ 0.4 от всей суммы
- СИС 0.3 от всей суммы
- ЦАТ 0.3 от всей суммы  

Причем для каждого портфеля, кол-во клиентов (клиенты могут повторяться) должно быть одинаково  
* Погрешность в сумме не должна превышать 3000

In [40]:
df = pd.read_excel('/content/vseKAtest.xlsx')

In [41]:
df = df.iloc[:,1:3]

In [42]:
df.head()

,keyPart,SumCred
0,3489395,1000
1,3092167,1000
2,2515372,1127
3,3189430,1270
4,3411061,1270


In [44]:
"Задаем сумму для портфелей"

x1 = {"ЩЭВ" : df['SumCred'].sum() * 0.4}
x2 = {"СИС" : df['SumCred'].sum() * 0.3}
x3 = {"ЦАТ" : df['SumCred'].sum() * 0.3}

list_bag = [x1, x2, x3]

In [45]:
"Число keypart для одного портфеля"
sum_keypart = 3
one_sum_keypart = int(round(len(df['keyPart']) / sum_keypart,0))
print(one_sum_keypart)

556


Всего должно быть 556 клиентов в каждом портфеле

In [46]:
def gek(one_sum_keypart, bag, df):
  "Подбор небходимых keypart по условиям для 1 портфеля"

  m = GEKKO()

  #define parameter
  eq = m.Param()
  m.options.SOLVER = 1
  m.options.MAX_ITER = 25

  #initialize variables
  x = m.Array(m.Var,len(df),integer=True,lb=0,ub=1) 

  #Equations
  m.Equation(m.sum(x) == one_sum_keypart)
  m.Equation(m.sum([df.iloc[i][1] * x[i] for i in range(len(df))]) <= list(bag.values())[0] + 1500)
  m.Equation(m.sum([df.iloc[i][1] * x[i] for i in range(len(df))]) >= list(bag.values())[0] - 1500)

  #Objective
  m.Maximize(m.sum([df.iloc[i][1] * x[i] for i in range(len(df))]))

  #Solve simulation
  m.solve()

  return x #бинарный вывод, где 1 - keypart включается

In [47]:
def save_table_bag(name_bag, df, x):
  "Обработка каждой таблицы портфеля"
  df[list(name_bag)[0]] = x
  df[list(name_bag)[0]] = df[list(name_bag)[0]].map(lambda x: x[0])
  df_agency = df[df[list(name_bag)[0]] == 1].iloc[:,:2]
  df_agency['Agency'] = list(name_bag)[0]
  df = df[df[list(name_bag)[0]] == 0].iloc[:,:2].reset_index(drop = True)
  return df_agency, df 

In [48]:
list_bag_table = []
for i in range(len(list_bag)):
  name_bag = list_bag[i]
  x = gek(one_sum_keypart, name_bag, df)
  df_table, df = save_table_bag(name_bag, df, x) #выводит таблицу для одного портфеля и измененный df (без keypart для первого портфеля)
  list_bag_table.append(df_table)

apm 34.125.45.47_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            4
   Constants    :            0
   Variables    :         6679
   Intermediates:            0
   Connections  :         6676
   Equations    :         5008
   Residuals    :         5008
 
 Number of state variables:           6678
 Number of total equations: -         5011
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :           1665
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      4.69 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -8.15E+06 Gap:       NaN
Iter:     2 I: -1 Tm:      0.09 NLPi:

In [56]:
"Проверка сходимости для каждого портфеля"
print('ЩЭВ')
print('Сходимость по клиентам:', len(list_bag_table[0]))
print('Отклонение по сумме:',list_bag_table[0]['SumCred'].sum() - list(list_bag[0].values())[0])
print('СИС')
print('Сходимость по клиентам:', len(list_bag_table[1]))
print('Отклонение по сумме:', list_bag_table[1]['SumCred'].sum() - list(list_bag[1].values())[0], 'СИС')
print('ЦАТ')
print('Сходимость по клиентам:', len(list_bag_table[2]))
print('Отклонение по сумме:', list_bag_table[2]['SumCred'].sum() - list(list_bag[2].values())[0], 'ЦАТ')

ЩЭВ
Сходимость по клиентам: 556
Отклонение по сумме: 484.59999999962747
СИС
Сходимость по клиентам: 556
Отклонение по сумме: 847.7000000001863 СИС
ЦАТ
Сходимость по клиентам: 556
Отклонение по сумме: -1332.2999999998137 ЦАТ


Таблицы с портфелями:

In [50]:
list_bag_table[0].head() #таблица для ЩЭВ

,keyPart,SumCred,Agency
523,3481495,6000,ЩЭВ
661,3093703,6545,ЩЭВ
675,3037410,6857,ЩЭВ
760,3293675,7356,ЩЭВ
765,3407704,7372,ЩЭВ


In [51]:
list_bag_table[1].head() #таблица для СИС

,keyPart,SumCred,Agency
0,3489395,1000,СИС
193,3323359,4607,СИС
194,3283343,4607,СИС
195,3432199,4607,СИС
197,3455217,4659,СИС


In [52]:
list_bag_table[2].head() #таблица для ЦАТ

,keyPart,SumCred,Agency
0,3092167,1000,ЦАТ
1,2515372,1127,ЦАТ
2,3189430,1270,ЦАТ
3,3411061,1270,ЦАТ
4,1956699,1417,ЦАТ
